# Advent of Code, 1st Dec 2024

## Historian Hysteria - Part 1

The Chief Historian is always present for the big Christmas sleigh launch, but nobody has seen him in months! Last anyone heard, he was visiting locations that are historically significant to the North Pole; a group of Senior Historians has asked you to accompany them as they check the places they think he was most likely to visit.

As each location is checked, they will mark it on their list with a star. They figure the Chief Historian must be in one of the first fifty places they'll look, so in order to save Christmas, you need to help them get fifty stars on their list before Santa takes off on December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the Advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

You haven't even left yet and the group of Elvish Senior Historians has already hit a problem: their list of locations to check is currently empty. Eventually, someone decides that the best place to check first would be the Chief Historian's office.

Upon pouring into the office, everyone confirms that the Chief Historian is indeed nowhere to be found. Instead, the Elves discover an assortment of notes and lists of historically significant locations! This seems to be the planning the Chief Historian was doing before he left. Perhaps these notes can be used to determine which locations to search?

Throughout the Chief's office, the historically significant locations are listed not by name but by a unique number called the location ID. To make sure they don't miss anything, The Historians split into two groups, each searching the office and trying to create their own complete list of location IDs.

There's just one problem: by holding the two lists up side by side (your puzzle input), it quickly becomes clear that the lists aren't very similar. Maybe you can help The Historians reconcile their lists?

For example:

| List one | List two |
|----------|----------|
| 3 | 4 |
| 4 | 3 |
| 2 | 5 |
| 1 | 3 |
| 3 | 9 |
| 3 | 3 |

Maybe the lists are only off by a small amount! To find out, pair up the numbers and measure how far apart they are. Pair up the smallest number in the left list with the smallest number in the right list, then the second-smallest left number with the second-smallest right number, and so on.

Within each pair, figure out how far apart the two numbers are; you'll need to add up all of those distances. For example, if you pair up a 3 from the left list with a 7 from the right list, the distance apart is 4; if you pair up a 9 with a 3, the distance apart is 6.

In the example list above, the pairs and distances would be as follows:

The smallest number in the left list is 1, and the smallest number in the right list is 3. The distance between them is 2.
The second-smallest number in the left list is 2, and the second-smallest number in the right list is another 3. The distance between them is 1.
The third-smallest number in both lists is 3, so the distance between them is 0.
The next numbers to pair up are 3 and 4, a distance of 1.
The fifth-smallest numbers in each list are 3 and 5, a distance of 2.
Finally, the largest number in the left list is 4, while the largest number in the right list is 9; these are a distance 5 apart.
To find the total distance between the left list and the right list, add up the distances between all of the pairs you found. In the example above, this is 2 + 1 + 0 + 1 + 2 + 5, a total distance of 11!

Your actual left and right lists contain many location IDs. What is the total distance between your lists?

### Read in the input data

In [5]:
#!F#
let inputFilePath = "aoc-2024-12-01-puzzle-input.txt"

Read the lines from the file.

In [6]:
open System.IO

let lines: string list = File.ReadAllLines(inputFilePath) |> Array.toList
lines[0]

58990   83989

Convert each line into a pair of integers.

In [7]:
let locationsPairs: (int*int) list =
    lines
    |> List.map (fun line -> line.Split(' ') |> Array.toList)
    |> List.map (fun strList -> (strList |> List.filter (fun str -> str.Length >= 1)))
    |> List.map (fun strList -> (strList |> List.map (fun str -> Int32.Parse(str))))
    |> List.map (fun pairList -> (pairList[0], pairList[1]))
locationsPairs[0]

Item1,58990
Item2,83989


Next we need to split the pairs into two lists, and sort those lists from smallest to largest.

In [14]:
let locations1, locations2 = locationsPairs |> List.unzip
let sortedLocations1 = locations1 |> List.sort
let sortedLocations2 = locations2 |> List.sort
let sortedLocationPairs: (int*int) list = List.zip sortedLocations1 sortedLocations2
sortedLocationPairs[0]

Item1,10173
Item2,10181


Now calculate the absolute difference of each pair, then sum those differences.

In [15]:
let distance = sortedLocationPairs |> List.map (fun (x:int,y:int) -> Math.Abs(x-y)) |> List.sum
distance

2113135

## Historian Hysteria - Part 2

Your analysis only confirmed what everyone feared: the two lists of location IDs are indeed very different.

Or are they?

The Historians can't agree on which group made the mistakes or how to read most of the Chief's handwriting, but in the commotion you notice an interesting detail: a lot of location IDs appear in both lists! Maybe the other numbers aren't location IDs at all but rather misinterpreted handwriting.

This time, you'll need to figure out exactly how often each number from the left list appears in the right list. Calculate a total similarity score by adding up each number in the left list after multiplying it by the number of times that number appears in the right list.

Here are the same example lists again:

| List one | List two |
|----------|----------|
| 3 | 4 |
| 4 | 3 |
| 2 | 5 |
| 1 | 3 |
| 3 | 9 |
| 3 | 3 |

For these example lists, here is the process of finding the similarity score:

The first number in the left list is 3. It appears in the right list three times, so the similarity score increases by 3 * 3 = 9.
The second number in the left list is 4. It appears in the right list once, so the similarity score increases by 4 * 1 = 4.
The third number in the left list is 2. It does not appear in the right list, so the similarity score does not increase (2 * 0 = 0).
The fourth number, 1, also does not appear in the right list.
The fifth number, 3, appears in the right list three times; the similarity score increases by 9.
The last number, 3, appears in the right list three times; the similarity score again increases by 9.
So, for these example lists, the similarity score at the end of this process is 31 (9 + 4 + 0 + 0 + 9 + 9).

Once again consider your left and right lists. What is their similarity score?

In [17]:
let rec findSimilarities (sortedList1: int list) (sortedList2: int list) (similarityMap: Map<int,int>): Map<int,int> =
    match sortedList1 with
    | [] -> similarityMap
    | head1::tail1 ->
        match sortedList2 with
        [] -> similarityMap
        | head2::tail2 ->
            match (head1, head2) with
            | (head1, head2) when head1 < head2 -> findSimilarities tail1 sortedList2 similarityMap
            | (head1, head2) when head1 > head2 -> findSimilarities sortedList1 tail2 similarityMap
            | (head1, head2) -> // head1 == head2
                let newSimilarityMap =
                    if (similarityMap |> Map.containsKey head1)
                    then similarityMap |> Map.add head1 (similarityMap[head1] + 1)
                    else similarityMap |> Map.add head1 1
                findSimilarities sortedList1 tail2 newSimilarityMap // don't discard 'head1' here

In [19]:
let similarities = findSimilarities sortedLocations1 sortedLocations2 Map.empty<int,int>
similarities

key,value
10311,12
13469,13
13732,1
17840,14
20464,18
22168,5
22708,8
25790,11
25959,17
26012,10


Having found the similarities, we now sum them.

In [20]:
similarities |> Map.toList |> List.map (fun (value,count) -> value*count) |> List.sum

19097157